In [5]:
import polars as pl

# Create a DataFrame
df = pl.DataFrame({
    "category": ["A", "A", "A", "A", "B", "B", "B", "B"],
    "item": ["Item1", "Item2", "Item3", "Item4", "Item5", "Item6", "Item7", "Item8"],
    "sales_amount": [100, 200, 150, 120, 300, 250, 200, 100]
})

print(df)

shape: (8, 3)
┌──────────┬───────┬──────────────┐
│ category ┆ item  ┆ sales_amount │
│ ---      ┆ ---   ┆ ---          │
│ str      ┆ str   ┆ i64          │
╞══════════╪═══════╪══════════════╡
│ A        ┆ Item1 ┆ 100          │
│ A        ┆ Item2 ┆ 200          │
│ A        ┆ Item3 ┆ 150          │
│ A        ┆ Item4 ┆ 120          │
│ B        ┆ Item5 ┆ 300          │
│ B        ┆ Item6 ┆ 250          │
│ B        ┆ Item7 ┆ 200          │
│ B        ┆ Item8 ┆ 100          │
└──────────┴───────┴──────────────┘


In [9]:
# we want the top 3 items by sales_amount per category
# the corresponding SQL query would be:
# 
# WITH RankedSales AS (
#     SELECT 
#         category,
#         item,
#         sales_amount,
#         ROW_NUMBER() OVER (PARTITION BY category ORDER BY sales_amount DESC) AS rn
#     FROM 
#         sales
# )
# SELECT 
#     category,
#     item,
#     sales_amount
# FROM 
#     RankedSales
# WHERE 
#     rn <= 3
# ORDER BY 
#     category, 
#     sales_amount DESC;

df.with_columns(
        pl.col('sales_amount')
            .rank('dense', descending=True)
            .over('category')
            .alias('rank')
    )\
    .filter(pl.col('rank') <= 3)\
    .sort(['category', 'rank'])


category,item,sales_amount,rank
str,str,i64,u32
"""A""","""Item2""",200,1
"""A""","""Item3""",150,2
"""A""","""Item4""",120,3
"""B""","""Item5""",300,1
"""B""","""Item6""",250,2
"""B""","""Item7""",200,3


In [12]:
pd_df = df.to_pandas()


pd_df['rank'] = pd_df.groupby('category')['sales_amount'].rank(method='dense', ascending=False)
# note that we needed to do an imperative modification of the DataFrame, which is clunky

# Also the syntax below is unintuitive and somewhat confusing / verbose
top_3_per_category = pd_df[pd_df['rank'] <= 3]

print(top_3_per_category)

  category   item  sales_amount  rank
1        A  Item2           200   1.0
2        A  Item3           150   2.0
3        A  Item4           120   3.0
4        B  Item5           300   1.0
5        B  Item6           250   2.0
6        B  Item7           200   3.0


In [11]:
top_3_per_category = (
    df.group_by("category", maintain_order=True)
      .agg([
          pl.col("item").sort_by("sales_amount", descending=True).head(3).alias("item"),
          pl.col("sales_amount").sort_by("sales_amount", descending=True).head(3).alias("sales_amount")
      ])
      .explode(["item", "sales_amount"]) # exploding on multiple columns, will correspond the values of the same index
)

print(top_3_per_category)

shape: (6, 3)
┌──────────┬───────┬──────────────┐
│ category ┆ item  ┆ sales_amount │
│ ---      ┆ ---   ┆ ---          │
│ str      ┆ str   ┆ i64          │
╞══════════╪═══════╪══════════════╡
│ A        ┆ Item2 ┆ 200          │
│ A        ┆ Item3 ┆ 150          │
│ A        ┆ Item4 ┆ 120          │
│ B        ┆ Item5 ┆ 300          │
│ B        ┆ Item6 ┆ 250          │
│ B        ┆ Item7 ┆ 200          │
└──────────┴───────┴──────────────┘
